<a href="https://colab.research.google.com/github/dhrits/mec2-projects/blob/main/DSagar_Student_MLE_MiniProject_Recommendation_Engines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Recommendation Engines

Recommendation engines are algorithms designed to provide personalized suggestions or recommendations to users. These systems analyze user behavior, preferences, and interactions with items (products, movies, music, articles, etc.) to predict and offer items that users are likely to be interested in. Recommendation engines play a crucial role in enhancing user experience, driving engagement, and increasing conversion rates in various applications, including e-commerce, entertainment, content platforms, and more.

There are generally two approaches taken in collaborative filtering and content-based recommendation engines:

**1. Collaborative Filtering:**
Collaborative Filtering is a popular approach to building recommendation systems that leverages the collective behavior of users to make personalized recommendations. It is based on the idea that users who have agreed in the past will likely agree in the future. There are two main types of collaborative filtering:

- **User-based Collaborative Filtering:** This method finds users similar to the target user based on their past interactions (e.g., ratings or purchases). It then recommends items that similar users have liked but the target user has not interacted with yet.

- **Item-based Collaborative Filtering:** In this approach, the system identifies similar items based on user interactions. It recommends items that are similar to the ones the target user has already liked or interacted with.

Collaborative filtering does not require any explicit information about items but relies on the similarity between users or items. It is effective in capturing complex patterns and can provide serendipitous recommendations. However, it suffers from the cold-start problem (i.e., difficulty in recommending to new users or items with no interactions) and scalability challenges in large datasets.

**2. Content-Based Recommendation:**
Content-based recommendation is an alternative approach to building recommendation systems that focuses on the attributes or features of items and users. It leverages the characteristics of items to make recommendations. The key steps involved in content-based recommendation are:

- **Feature Extraction:** For each item, relevant features are extracted. For movies, these features could be genre, director, actors, and plot summary.

- **User Profile:** A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.

- **Similarity Calculation:** The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity or Euclidean distance.

- **Recommendation:** Items that are most similar to the user profile are recommended to the user.

Content-based recommendation systems are less affected by the cold-start problem as they can still recommend items based on their features. They are also more interpretable as they rely on item attributes. However, they may miss out on providing serendipitous recommendations and can be limited by the quality of feature extraction and user profiles.

**Choosing Between Collaborative Filtering and Content-Based:**
Both collaborative filtering and content-based approaches have their strengths and weaknesses. The choice between them depends on the specific requirements of the recommendation system, the type of data available, and the user base. Hybrid approaches that combine collaborative filtering and content-based techniques are also common, aiming to leverage the strengths of both methods and mitigate their weaknesses.

In this mini-project, you'll be building both content based and collaborative filtering engines for the [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/25m/). The MovieLens 25M dataset is one of the most widely used and popular datasets for building and evaluating recommendation systems. It is provided by the GroupLens Research project, which collects and studies datasets related to movie ratings and recommendations. The MovieLens 25M dataset contains movie ratings and other related information contributed by users of the MovieLens website.

**Dataset Details:**
- **Size:** The dataset contains approximately 25 million movie ratings.
- **Users:** It includes ratings from over 162,000 users.
- **Movies:** The dataset consists of ratings for more than 62,000 movies.
- **Ratings:** The ratings are provided on a scale of 1 to 5, where 1 is the lowest rating and 5 is the highest.
- **Timestamps:** Each rating is associated with a timestamp, indicating when the rating was given.

**Data Files:**
The dataset is usually split into three CSV files:

1. **movies.csv:** Contains information about movies, including the movie ID, title, genres, and release year.
   - Columns: movieId, title, genres

2. **ratings.csv:** Contains movie ratings provided by users, including the user ID, movie ID, rating, and timestamp.
   - Columns: userId, movieId, rating, timestamp

3. **tags.csv:** Contains user-generated tags for movies, including the user ID, movie ID, tag, and timestamp.
   - Columns: userId, movieId, tag, timestamp

First, import all the libraries you'll need.

In [ ]:
import zipfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.metrics.pairwise import cosine_similarity

Next, download the relevant components of the MoveLens dataset. Note, these instructions are roughly based on the colab [here](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=O3bcgduFo4s6).

In [ ]:
print("Downloading movielens data...")

urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'movielens.zip')
zip_ref = zipfile.ZipFile('movielens.zip', 'r')
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


Before doing any kind of machine learning, it's always good to familiarize yourself with the datasets you'lll be working with.

Here are your tasks:

1. Spend some time familiarizing yourself with both the `movies` and `ratings` dataframes. How many unique user ids are present? How many unique movies are there?
2. Create a new dataframe that merges the `movies` and `ratings` tables on 'movie_id'. Only keep the 'user_id', 'title', 'rating' fields in this new dataframe.

In [ ]:
# Spend some time familiarizing yourself with both the movies and ratings
# dataframes. How many unique user ids are present? How many unique movies
# are there?

In [ ]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
print(f"Number of unique user ids: {ratings['user_id'].nunique()}")
print(f"Number of unique movie ids: {ratings['movie_id'].nunique()}")
print(f"Shape of ratings dataframe: {ratings.shape}")

Number of unique user ids: 943
Number of unique movie ids: 1682
Shape of ratings dataframe: (100000, 4)


There are **943 unique user ids** and **100000 user/movie** ratings.

In [ ]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
movies.drop_duplicates(subset='title', inplace=True)

In [ ]:
print(f'Size of movies: {movies.shape}')
print(f"Unique movies: {movies['movie_id'].nunique()}")
print(f"Number of unique genres: {len(movies.drop(columns=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']).columns)}")

Size of movies: (1664, 24)
Unique movies: 1664
Number of unique genres: 19


There are 1682 unique movies in the dataset described using 19 genres.

In [ ]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
# Merge movies and ratings dataframes

In [ ]:
movie_ratings = movies.merge(ratings, on='movie_id')[['user_id', 'title', 'rating']]
movie_ratings.drop_duplicates(subset=['user_id', 'title'], inplace=True)

In [ ]:
print(f'Shape of movies: {movie_ratings.shape}')
print(f"Unique movies: {movies['movie_id'].nunique()}")

Shape of movies: (99392, 3)
Unique movies: 1664


As mentioned in the introduction, content-Based Filtering is a recommendation engine approach that focuses on the attributes or features of items (products, movies, music, articles, etc.) and leverages these features to make personalized recommendations. The underlying idea is to match the characteristics of items with the preferences of users to suggest items that align with their interests. Content-based filtering is particularly useful when explicit user-item interactions (e.g., ratings or purchases) are sparse or unavailable.

**Key Steps in Content-Based Filtering:**

1. **Feature Extraction:**
   - For each item, relevant features are extracted. These features are typically descriptive attributes that can be represented numerically, such as genre, director, actors, author, publication date, and keywords.
   - In the case of text-based items, natural language processing techniques may be used to extract features like TF-IDF (Term Frequency-Inverse Document Frequency) scores.

2. **User Profile Creation:**
   - A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.
   - For example, if a user has watched several action movies, the action genre feature would receive a higher weight in their profile.

3. **Similarity Calculation:**
   - The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity, Euclidean distance, or Pearson correlation.
   - Cosine similarity is commonly used as it measures the cosine of the angle between two vectors, which represents their similarity.

4. **Recommendation:**
   - Items that are most similar to the user profile are recommended to the user. These are items whose features have the highest similarity scores with the user profile.
   - The recommended items are presented as a list sorted by their similarity scores.

**Advantages of Content-Based Filtering:**
1. **No Cold-Start Problem:** Content-based filtering can make recommendations even for new users with no historical interactions because it relies on item features rather than user history.

2. **User Independence:** The recommendations are based solely on the features of items and do not require knowledge of other users' preferences or behavior.

3. **Transparency:** Content-based recommendations are interpretable, as they depend on the features of items, making it easier for users to understand why specific items are recommended.

4. **Serendipity:** Content-based filtering can recommend items with characteristics not seen before by the user, leading to serendipitous discoveries.

5. **Diversity in Recommendations:** The method can offer diverse recommendations since it suggests items with different feature combinations.

**Limitations of Content-Based Filtering:**
1. **Limited Discovery:** Content-based filtering may struggle to recommend items outside the scope of users' historical interactions or interests.

2. **Over-Specialization:** Users may receive recommendations that are too similar to their previous choices, leading to a lack of exposure to new item categories.

3. **Dependency on Feature Quality:** The quality and relevance of item features significantly influence the quality of recommendations.

4. **Limited for Cold Items:** Content-based filtering can struggle to recommend new items with limited feature information.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return content-based recommendations for this user. Here are steps you can take:

  A. Get the user's rated movies

  B. Create a TF-IDF matrix using movie genres. Note, this can be extracted from the `movies` dataframe.

  C. Compute the cosine similarity between movie genres. Use the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function.

  D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.

  E. Remove duplicates and movies already rated by the user.

In [ ]:
# Helper method for extracting "tfidf matrix"
def get_tfidf_df(movies):
  movies_df = movies[['title'] + genre_cols]
  movies_df = movies_df.set_index('title')
  data = movies_df.values
  # Extract term frequency
  data_tf = data/float(len(genre_cols))
  # extract inverse document frequency
  data_idf = np.log(data.shape[0]/(1 + data.sum(0)))
  data_tfidf = data_tf * data_idf
  movies_df = pd.DataFrame(data_tfidf, index=movies_df.index, columns=movies_df.columns)
  return movies_df

# Content-Based Filtering using Movie Genres
def content_based_recommendation(user_id, df, nrecs=5):
  # Get the user's rated movies
  rated_movies = df[df['user_id'] == user_id]['title']
  # Create a TF-IDF matrix using movie genres
  movies_tfidf = get_tfidf_df(movies)
  # Compute the cosine similarity between movie genres
  user_profile = movies_tfidf.reindex(rated_movies.drop_duplicates())
  user_profile = user_profile.mean(axis=0)
  sims = cosine_similarity(user_profile.values.reshape(1, -1), movies_tfidf.values)
  # Get the indices of the similar movies based on cosine similarity
  indices = np.argsort(-sims)
  # Remove duplicates and movies already rated by the user
  sorted_recs = pd.Series(movies.title.values[indices].reshape(-1))
  sorted_recs = sorted_recs[~sorted_recs.isin(rated_movies)].values
  # Please note that I deliberately filter to 5 here instead of while generating
  # the unfiltered list to ensure user gets some recommendations
  return sorted_recs[:nrecs]

The key idea behind collaborative filtering is that users who have agreed in the past will likely agree in the future. Instead of relying on item attributes or user profiles, collaborative filtering identifies patterns of user behavior and item preferences from the interactions present in the data.

**Types of Collaborative Filtering:**
There are two main types of collaborative filtering:

**Collaborative Filtering Process:**
The collaborative filtering process typically involves the following steps:

1. **Data Collection:**
   - Gather data on user-item interactions, such as movie ratings, product purchases, or article clicks.

2. **User-Item Matrix:**
   - Organize the data into a user-item matrix, where rows represent users, columns represent items, and the entries contain the users' interactions (e.g., ratings).

3. **Similarity Calculation:**
   - Calculate the similarity between users or items using similarity metrics such as cosine similarity, Pearson correlation, or Jaccard similarity.
   - For user-based collaborative filtering, user similarities are calculated, and for item-based collaborative filtering, item similarities are calculated.

4. **Neighborhood Selection:**
   - For each user or item, select the most similar users or items as the neighborhood.
   - The size of the neighborhood (the number of similar users or items to consider) is an important parameter to control the system's behavior.

5. **Prediction Generation:**
   - Predict the ratings for items that the target user has not yet interacted with by combining the ratings of neighboring users or items.

6. **Recommendation Generation:**
   - Recommend items with the highest predicted ratings to the target user.

**Advantages of Collaborative Filtering using User-Item Interactions:**
- Collaborative filtering is based solely on user interactions and does not require knowledge of item attributes, making it useful for cases where item data is sparse or unavailable.
- It can provide serendipitous recommendations, suggesting items that users may not have discovered on their own.
- Collaborative filtering can be applied in various domains, including e-commerce, music, movie, and content recommendations.

**Limitations of Collaborative Filtering:**
- The cold-start problem: Collaborative filtering struggles to recommend to new users or items with no or limited interaction history.
- It may suffer from sparsity when data is limited or when users have only interacted with a small subset of items.
- Scalability issues can arise with large datasets and an increasing number of users or items.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return collaborative filtering recommendations for this user based on a user-item interaction matrix. Here are steps you can take:

  A. Create the user-item matrix using Pandas' [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

  B. Fill missing values with zeros in this matrix.

  C. Calculate user-user similarity matrix using cosine similarity.

  D. Get the array of similarity scores of the target user with all other users from the similarity matrix.

  E. Extract, say the the top 5 most similar users (excluding the target user).

  F. Generate movie recommendations based on the most similar users.

  G. Remove duplicate movies recommendations.

In [ ]:
# Collaborative Filtering using User-Item Interactions
def collaborative_filtering_recommendation(user_id, df, nrecs=5):
  user_items = df.pivot_table(index='user_id', columns='title', values='rating')
  avg_ratings = user_items.mean(axis=1)
  user_items = user_items.sub(avg_ratings, axis=0)
  #Fill missing values with 0 (indicating no rating)
  user_items.fillna(0, inplace=True)
  # Calculate user-user similarity matrix using cosine similarity
  similarities = pd.DataFrame(cosine_similarity(user_items.values),
                              index=user_items.index,
                              columns=user_items.index)
  # Get the similarity scores of the target user with all other users
  similarity = similarities.loc[user_id].values
  # Find the top N most similar users (excluding the target user)
  sorted_idxs = np.argsort(-similarity)
  top_user_matches = similarities.index[sorted_idxs][1:6]
  # Generate movie recommendations based on the most similar users
  # Reindex user_items based on similar users
  recs = user_items.reindex(top_user_matches)
  # Get predicted movie scores based on closest users
  recs = recs.mean(axis=0)
  # Get list of rated movies for user_id
  rated = df[df['user_id'] == user_id]['title']
  unrated_mask = ~user_items.columns.isin(rated)
  # Set predicted scores for movies not yet rated by the user
  user_items.loc[user_id][unrated_mask] = recs[unrated_mask]
  # Sort recommendations based on highest score
  sorted_recs_idxs = np.argsort(-user_items.loc[user_id])
  sorted_recs = user_items.columns[sorted_recs_idxs.values]
  # Remove duplicates from recommendations
  top_recs = sorted_recs[~sorted_recs.isin(rated)][:nrecs].values
  return top_recs

Now, test your recommendations engines! Select a few user ids and generate recommendations using both functions you've written. Are the recommendations similar? Do the recommendations make sense?

## Testing Collaborative Filtering

In [ ]:
# Test the recommendation engines

# Helper method to get user profile
def get_user_profile(user_id):
  tfidf = get_tfidf_df(movies)
  rated = movie_ratings[movie_ratings['user_id'] == user_id]['title']
  user_profile = movies_tfidf.reindex(rated.drop_duplicates())
  user_profile = user_profile.mean(axis=0)
  return user_profile


def get_most_similar_users(user_id):
  user_items = movie_ratings.pivot_table(index='user_id', columns='title', values='rating')
  avg_ratings = user_items.mean(axis=1)
  user_items = user_items.sub(avg_ratings, axis=0)
  #Fill missing values with 0 (indicating no rating)
  user_items.fillna(0, inplace=True)
  # Calculate user-user similarity matrix using cosine similarity
  similarities = pd.DataFrame(cosine_similarity(user_items.values),
                              index=user_items.index,
                              columns=user_items.index)
  # Get the similarity scores of the target user with all other users
  similarity = similarities.loc[user_id].values
  # Find the top N most similar users (excluding the target user)
  sorted_idxs = np.argsort(-similarity)
  top_user_matches = similarities.index[sorted_idxs][1:6]
  return top_user_matches

Let's first test the list of suggestions provided by **collaborative filtering**. In order to test it, we'll randomly use sample user **244**

In [ ]:
user_id = 244

In [ ]:
# Getting content recs for user 244
cf_recs = collaborative_filtering_recommendation(user_id, movie_ratings)
cf_recs

array(['Casablanca (1942)', 'Godfather, The (1972)',
       'Cinema Paradiso (1988)', 'Silence of the Lambs, The (1991)',
       'Seven (Se7en) (1995)'], dtype=object)

To validate these recommendations, let's first see what user 244 has rated highly

In [ ]:
user_ratings = movie_ratings[movie_ratings['user_id'] == user_id]
highly_rated_by_user = user_ratings[user_ratings['rating'] >= 3]
highly_rated_by_user

,user_id,title,rating
197,244,Toy Story (1995),4
613,244,Four Rooms (1995),5
1004,244,Twelve Monkeys (1995),4
1742,244,Dead Man Walking (1995),5
2511,244,Mighty Aphrodite (1995),4
...,...,...,...
96284,244,Major Payne (1994),3
96423,244,Young Guns II (1990),4
96731,244,Mixed Nuts (1994),3
98614,244,SubUrbia (1997),4


Now let's find the closest users to user_id and see the movies they rated most highly.

In [ ]:
users = get_most_similar_users(user_id)

In [ ]:
close_users_ratings = movie_ratings[movie_ratings.user_id.isin(users)]
highly_rated_by_close_users = close_users_ratings[close_users_ratings['rating'] >= 3]
highly_rated_by_close_users

,user_id,title,rating
35,94,Toy Story (1995),4
348,916,Toy Story (1995),4
359,854,Toy Story (1995),3
374,838,Toy Story (1995),5
462,213,GoldenEye (1995),4
...,...,...,...
98410,916,M. Butterfly (1993),3
98611,916,SubUrbia (1997),3
99248,916,Romper Stomper (1992),3
99387,854,Sweet Nothing (1995),3


Now we can check how many of the movies the user has rated highly are in the movies that the list of close users have rated highly

In [ ]:
print(f"Percentage overlap in movies highly rated by user: {100 * highly_rated_by_user['title'].isin(highly_rated_by_close_users['title']).values.sum()/float(len(highly_rated_by_user)):.3f}")

Percentage overlap in movies highly rated by user: 75.253


So ~75% of movies rated highly by the sample user are also rated highly by the selected similar users.

Now let's check if the list of recommendations is in the list of movies rated highly by similar users.

In [ ]:
all(pd.Series(cf_recs).isin(highly_rated_by_close_users['title']))

True

In [ ]:
# We can write the above as a test
def test_collaborative_filtering(user_id, movie_ratings):
  cf_recs = collaborative_filtering_recommendation(user_id, movie_ratings)
  user_ratings = movie_ratings[movie_ratings['user_id'] == user_id]
  highly_rated_by_user = user_ratings[user_ratings['rating'] >= 3]
  assert(highly_rated_by_user['title'].isin(highly_rated_by_close_users['title']).values.sum()/float(len(highly_rated_by_user)) > 0.5)
  assert((pd.Series(cf_recs).isin(highly_rated_by_close_users['title'])).sum()/float(len(cf_recs)) > 0.5)

In [ ]:
test_collaborative_filtering(196, movie_ratings)
test_collaborative_filtering(76, movie_ratings)
test_collaborative_filtering(38, movie_ratings)
test_collaborative_filtering(127, movie_ratings)

## Testing Content Based Recommendations

Once again lets use sample user_id 244 to walk through the process

In [ ]:
user_id = 244

In [ ]:
content_recs = content_based_recommendation(user_id, movie_ratings)
content_recs

array(['American President, The (1995)', 'Wings of Desire (1987)',
       'What Happened Was... (1994)', 'Corrina, Corrina (1994)',
       'Don Juan DeMarco (1995)'], dtype=object)

It is obvious that these recommendations are quite different from collaborative filtering based recommendations. This is somewhat expected because while collaborative filtering was user-based, this approach is based on item attributes. Let's compare these movies to the user profile.

In [ ]:
user_profile = get_user_profile(user_id)
user_profile

genre_unknown    0.000000
Action           0.019784
Adventure        0.011189
Animation        0.007307
Children         0.004657
Comedy           0.026834
Crime            0.004859
Documentary      0.000774
Drama            0.018510
Fantasy          0.001902
Film-Noir        0.001865
Horror           0.005163
Musical          0.007493
Mystery          0.002937
Romance          0.020846
Sci-Fi           0.013689
Thriller         0.013077
War              0.015342
Western          0.003628
dtype: float64

In [ ]:
highly_rated_genres = user_profile[user_profile > 0.009].index
highly_rated_genres

Index(['Action', 'Adventure', 'Comedy', 'Drama', 'Romance', 'Sci-Fi',
       'Thriller', 'War'],
      dtype='object')

In [ ]:
movies_tfidf = get_tfidf_df(movies)

Let's look at the profile of the movies suggested to the user.

In [ ]:
movies_profile = movies_tfidf[movies_tfidf.index.isin(content_recs)].mean(axis=0)
movies_profile

genre_unknown    0.000000
Action           0.000000
Adventure        0.000000
Animation        0.000000
Children         0.000000
Comedy           0.062968
Crime            0.000000
Documentary      0.000000
Drama            0.044311
Fantasy          0.000000
Film-Noir        0.000000
Horror           0.000000
Musical          0.000000
Mystery          0.000000
Romance          0.100827
Sci-Fi           0.000000
Thriller         0.000000
War              0.000000
Western          0.000000
dtype: float64

Now let's look at the similarity between the profile of the suggested movies and the user's profile

In [ ]:
similarity = cosine_similarity(movies_profile.values.reshape(1, -1), user_profile.values.reshape(1, -1)).squeeze().item()
print(f"Similarity between user profile and suggested movies: {similarity: .3f}")

Similarity between user profile and suggested movies:  0.685


Overall, the similarity profile is quite high, however, it is also clear that the user's preference for Comedy and Romance is dominating these suggestions.

We can also refactor this into a test

In [ ]:
def test_content_based_recommendations(user_id, movie_ratings, movies):
  content_recs = content_based_recommendation(user_id, movie_ratings)
  user_profile = get_user_profile(user_id)
  movies_tfidf = get_tfidf_df(movies)
  movies_profile = movies_tfidf[movies_tfidf.index.isin(content_recs)].mean(axis=0)
  similarity = cosine_similarity(movies_profile.values.reshape(1, -1), user_profile.values.reshape(1, -1)).squeeze().item()
  assert(similarity > 0.5)

In [ ]:
test_content_based_recommendations(196, movie_ratings, movies)
test_content_based_recommendations(254, movie_ratings, movies)
test_content_based_recommendations(22, movie_ratings, movies)

Thus while the two methods don't necessarily produce the same recommendations, they are consistent in their methodology for producing recommendations.